In [41]:
import os
import pandas as pd
import openai
from openai import OpenAI, beta
import sys
import json
import pprint

sys.path.append(os.path.abspath(".."))
from src import config

from dotenv import load_dotenv
load_dotenv()

client = OpenAI(
  api_key=os.getenv('OPENAI_API_KEY')
)

from docling.document_converter import DocumentConverter

## El problema que falta resolver es vincular las facturas con el comprobante previo que corresponda:
- arrendamientos: cuota arr cargada en albor
- fletes: con carta de porte
- labores: con OT
- gastos vehiculos: con patente del vehiculo
- gastos comerciales: tipo servicio (ej: sellado, comisión, etc.)

In [24]:
from pydantic import BaseModel
from typing import Optional

from pydantic import BaseModel
from typing import Optional, List

class InvoiceItem(BaseModel):
    codigo: str
    descripcion: str
    unidad_medida: str
    cantidad: float
    precio_unitario: float
    importe: float
    remito: Optional[str] = None
    mes_servicio: Optional[str] = None

class InvoiceTax(BaseModel):
    concepto: str
    base_imponible: Optional[float] = None
    impuesto: str
    monto_impuesto: Optional[float] = None
    total: float

class Invoice(BaseModel):
    tipo: str  # Factura Original o Duplicado
    numero: str
    fecha: str
    cuit_emisor: str
    dgr_inscripto: str
    inicio_actividad: str
    razon_social_cliente: str
    categoria_iva_cliente: str
    domicilio_cliente: str
    cuit_cliente: str
    codigo_cliente: str
    condicion_venta: str
    pedido: str
    vendedor: str
    deposito: str
    moneda: str
    tipo_cambio: float
    items: List[InvoiceItem]
    impuestos: List[InvoiceTax]
    subtotal: float
    total: float
    observaciones: Optional[str] = None
    cae: str
    vencimiento_cae: str
    codigo_barras: str

class Invoices(BaseModel):
    invoices: list[Invoice] 

In [43]:
root_dir = config.EXTERNAL_DATA_DIR

In [ ]:
file_list = []

for root, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.pdf'):
            file_path = os.path.join(root, file)
            print(file_path)
            file_list.append(file_path)

In [27]:
query = "Eres un prolijo y laborioso data entry, que extae muy detalladamente los datos de las facturas, para pasarlos a una tabla con el formato dado."

In [ ]:
processed = []
dfs = []

for file_path in file_list:
    if file_path.endswith('.pdf'):
        print(f'Processing {file_path}')

        try:
            source = file_path # document per local path or URL
            converter = DocumentConverter()
            result = converter.convert(source)
            result_text = result.document.export_to_markdown()
        
        except Exception as e:
            print(f'Not processed: {file_path} - error: {e}')
            continue

        try:
            response = beta.chat.completions.parse(
                model="gpt-4o-mini-2024-07-18",
                messages=[
                    {
                        "role": "user",
                        "content": f"Los datos del archivo {file} están en formato markdown:\n{result_text}\n\nPregunta: {query}"
                    }
                ],
                temperature=0,
                max_tokens=15000,
                response_format=Invoice,
                top_p=1
            )

            # Extract the content from the response
            json_content = response.choices[0].message.content  # Accessing the attribute

            data = json.loads(json_content)

            print(data)

            # Convert to DataFrame
            df = pd.DataFrame(data)

            df['root'] = f'{root}'
            df['file'] = f'{file}'

            dfs.append(df)

            processed.append(file_path)

        except Exception as e:
            print(e)

if len(dfs) > 0:
    final_df = pd.concat(dfs)
else:
    final_df = pd.DataFrame()
    print('No dfs to concat')
    pass

In [31]:
# final_df

In [30]:
final_df.to_excel(f'{root_dir}/data_extraida.xlsx')